# Story Teller

In [50]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

Next, we configure the code by setting constants and hyper-parameters

In [64]:
K_NAME = 'namer'
K_HF_CARD_URL = 'url'
K_CHAT_PROMPT = 'is_chat_prompt'

model_configs = {
    'gpt2': {
        K_NAME: 'gpt2',
        K_HF_CARD_URL: 'https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct',
        K_CHAT_PROMPT: False
    },
    'llama-3-8b-instruct': {
        K_NAME: 'meta-llama/Meta-Llama-3-8B-Instruct',
        K_HF_CARD_URL: '',
        K_CHAT_PROMPT: True
    }
}

config = 'gpt2'

model_id = model_configs[config][K_NAME]
print(f'Using model: {model_id}')

Using model: gpt2


In [52]:
INPUT_TEXT = 'I enjoy walking with my cute dog'

In [63]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [62]:
# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    pad_token_id=tokenizer.eos_token_id)

model.device.type

'cpu'

In [65]:
# encode context the generation is conditioned on
if model_configs[config][K_CHAT_PROMPT]:
    messages = [
        {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
        {"role": "user", "content": INPUT_TEXT},
    ]
    tokens = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
else:
    tokens = tokenizer(INPUT_TEXT, return_tensors='pt').to(model.device.type)
tokens

{'input_ids': tensor([[   40,  2883,  6155,   351,   616, 13779,  3290]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [66]:
# generate 40 new tokens
output = model.generate(**tokens, max_new_tokens=40, output_attentions=True)
output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[   40,  2883,  6155,   351,   616, 13779,  3290,    11,   290,   314,
          1101,   257,  1263,  4336,   286,   262,  3290,  3952,    13,   314,
          1053,   587,   284,   262,  3290,  3952,   257,  1178,  1661,    11,
           290,   314,  1053,  1239,   587,   523,  6568,   284,   766,   262,
          3290,  3952,    13,   198,   198,    40,  1053]])

In [67]:
print("Output:\n" + 100 * '-')
print(tokenizer.decode(output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, and I'm a big fan of the dog park. I've been to the dog park a few times, and I've never been so excited to see the dog park.

I've
